# Salary Predictions
## Nate Borries
### June 2023

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [6]:
# read in data from csv file
df = pd.read_csv(r"C:\Users\Nate\Documents\DataPortfolio\Website1\Levels_Fyi_Salary_Data.csv")
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [8]:
# explore data set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  float64
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

In [11]:
# build new data frame
df.columns

Index(['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber', 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree',
       'Highschool', 'Some_College', 'Race_Asian', 'Race_White',
       'Race_Two_Or_More', 'Race_Black', 'Race_Hispanic', 'Race', 'Education'],
      dtype='object')

In [16]:
# use specific columns from above to revise df (salaryDF)
# assume largest correlation of salary from company, yearsofexperience, and yearsatcompany
salaryDF = df[['company','totalyearlycompensation','yearsofexperience','yearsatcompany','basesalary']]
salaryDF.head() # check to see new df

,company,totalyearlycompensation,yearsofexperience,yearsatcompany,basesalary
0,Oracle,127000,1.5,1.5,107000.0
1,eBay,100000,5.0,3.0,0.0
2,Amazon,310000,8.0,0.0,155000.0
3,Apple,372000,7.0,5.0,157000.0
4,Microsoft,157000,5.0,3.0,0.0


## Crate training and testing data

y = totalyearlycompensation # what we're trying to predict.


x1 = yearsofexperience
x2 = yearsatcompany
x3 = basesalary

simple linear regression equation...
y = w1*x1 + w2*x2 + w3*x3 + b

rewrite in matrix format... 
Y = W*X

X.T = (x1 x2 x3 1)

W.T = (w1 w2 w3 b)

In [32]:
x_train = salaryDF.iloc[:50000,2:] # taking first 50k rows of yearsofexperience, yearsatcompany, basesalary (2:)
y_train = salaryDF.iloc[:50000,1] # only need totalyearlycompensation for the same amount of rows.

x_test = salaryDF.iloc[50000:, 2:] # use rest of data to test model
y_test = salaryDF.iloc[50000:,1]

x_train.insert(loc = 3, column='b', value = 1) # need a column to put b values
x_test.insert(loc = 3, column='b', value = 1)


print(y_train)

0        127000
1        100000
2        310000
3        372000
4        157000
          ...  
49995     18000
49996    148000
49997    190000
49998    258000
49999     85000
Name: totalyearlycompensation, Length: 50000, dtype: int64


### Functions for RMSE & Calculating Weights

In [30]:
# make RSME funciton
def rmse(targets, predictions):
    return np.sqrt((np.square(predictions-targets)).mean())

# make function to calculate weights
def calculateWeights(x_train, y_train):
    w = np.linalg.inv(np.transpose(x_train).dot(x_train)).dot(np.transpose(x_train).dot(y_train)) # parameter estimation with least squares solution
    return w

In [34]:
w = calculateWeights(x_train, y_train)
w

array([ 5.37710127e+03, -1.49720232e+03,  1.24746924e+00,  1.36560933e+04])

In [40]:
# calculate training error by finding yvalue predictions
y_train_pred = x_train.dot(w)
train_error = rmse(y_train, y_train_pred)
print(train_error)

# testing error on data we havent used yet
y_test_pred = x_test.dot(w)
test_error = rmse(y_test, y_test_pred)
print(test_error)

# errors look very large! 
# might be caused by basesalary values of '0'

92469.43809773923
121382.52351504719


In [42]:
# try to predict yearly compensation based on random values picked by user.

new_pred = np.array([5, 3, 80000, 1]).dot(w)
print(new_pred)

135847.53227936703
